# 🎨 Trellis NIM - Simple Deployment & Inference

A streamlined notebook for deploying NVIDIA Trellis NIM and generating 3D models from text prompts.

## Steps:
1. **GPU Check** - Verify L40s GPU availability
2. **Deploy NIM** - Run the Trellis NIM container
3. **Generate 3D Models** - Simple prompt-based generation

---


## Step 1: Check GPU Availability

Verify that we have access to an L40s GPU:


In [ ]:
!nvidia-smi


## Step 2: Deploy Trellis NIM Container

**⚠️ IMPORTANT**: Replace `<PASTE_API_KEY_HERE>` with your actual NGC API key from https://ngc.nvidia.com/setup/api-key


In [ ]:
# Set your NGC API key here
import os
os.environ['NGC_API_KEY'] = '<PASTE_API_KEY_HERE>'  # Replace with your actual API key

# Verify the API key is set
if os.environ['NGC_API_KEY'] == '<PASTE_API_KEY_HERE>':
    print('❌ Please replace <PASTE_API_KEY_HERE> with your actual NGC API key!')
else:
    print('✅ NGC API key is set')


In [ ]:
# Docker login to NGC registry
!echo "$NGC_API_KEY" | docker login nvcr.io --username '$oauthtoken' --password-stdin


In [ ]:
# Create cache directory
!mkdir -p ~/.cache/nim
!chmod 777 ~/.cache/nim
!echo "Cache directory created at: ~/.cache/nim"


In [ ]:
# Start Trellis NIM container (this will run in the background)
# Note: This may take several minutes to download and start the first time

!docker run -d --name=nim-server \
  --runtime=nvidia --gpus='"device=0"' \
  -e NGC_API_KEY=$NGC_API_KEY \
  -p 8000:8000 \
  -v ~/.cache/nim:/opt/nim/.cache/ \
  nvcr.io/nim/microsoft/trellis:latest

print('🚀 Trellis NIM container started!')
print('⏳ Please wait 2-5 minutes for the container to fully initialize...')


In [ ]:
# Check container status
!docker ps | grep nim-server
!echo "\n📊 Container logs (last 10 lines):"
!docker logs nim-server --tail 10


## Step 3: Generate 3D Models

Now you can generate 3D models by simply changing the prompt below and running the cell:


In [ ]:
# 🎯 CHANGE THIS PROMPT TO GENERATE DIFFERENT 3D MODELS
PROMPT = "A simple coffee shop interior"

# Optional: Change the seed for different variations (0 = random)
SEED = 0

# Optional: Change output filename
OUTPUT_FILE = "result.glb"

print(f"🎨 Generating 3D model for: '{PROMPT}'")
print(f"💾 Output file: {OUTPUT_FILE}")
print(f"🎲 Seed: {SEED}")
print("⏳ Please wait...")


In [ ]:
# First, let's test if the NIM container is responding
print("🔍 Testing NIM container connectivity...")
health_check = !curl -s http://localhost:8000/health || curl -s http://localhost:8000/ || echo "CONNECTION_FAILED"

if "CONNECTION_FAILED" in ' '.join(health_check):
    print("❌ Cannot connect to NIM container at http://localhost:8000")
    print("💡 Make sure the container is running with: docker ps | grep nim-server")
    print("💡 Check container logs with: docker logs nim-server")
else:
    print("✅ NIM container is responding")

print("\n📡 Sending generation request to Trellis NIM...")

# Use the exact curl command format from your original request
import subprocess
import json
import base64

# Create the JSON payload
json_payload = json.dumps({
    "prompt": PROMPT,
    "seed": SEED
})

# Execute curl command exactly as specified
curl_cmd = [
    'curl', '-X', 'POST', 'http://localhost:8000/v1/infer',
    '-H', 'Accept: application/json',
    '-H', 'Content-Type: application/json',
    '-d', json_payload,
    '--silent'
]

print(f"🎯 Generating: '{PROMPT}'")
print("⏳ This may take 1-3 minutes...")

try:
    # Run the curl command
    result = subprocess.run(curl_cmd, capture_output=True, text=True, timeout=300)
    
    if result.returncode != 0:
        print(f"❌ Curl command failed with return code: {result.returncode}")
        print(f"Error: {result.stderr}")
        print(f"Command: {' '.join(curl_cmd)}")
    else:
        response_text = result.stdout.strip()
        print(f"📨 Raw response length: {len(response_text)} characters")
        print(f"📨 Response preview: {response_text[:300]}...")
        
        if not response_text:
            print("❌ Empty response from server")
        elif response_text.startswith('<!DOCTYPE') or '<html' in response_text:
            print("❌ Received HTML response instead of JSON - check if container is fully started")
        else:
            try:
                # Parse JSON response
                response_data = json.loads(response_text)
                print("✅ Valid JSON response received")
                
                # Check for artifacts in response
                if 'artifacts' in response_data and len(response_data['artifacts']) > 0:
                    artifact = response_data['artifacts'][0]
                    if 'base64' in artifact:
                        base64_data = artifact['base64']
                        
                        # Decode and save the GLB file
                        glb_data = base64.b64decode(base64_data)
                        
                        with open(OUTPUT_FILE, 'wb') as f:
                            f.write(glb_data)
                        
                        print(f"✅ SUCCESS! 3D model saved as: {OUTPUT_FILE}")
                        print(f"📊 File size: {len(glb_data) / 1024:.2f} KB")
                        print(f"🎯 You can now open {OUTPUT_FILE} in any 3D viewer or Blender!")
                    else:
                        print("❌ No base64 data in artifact")
                        print(f"Artifact keys: {list(artifact.keys())}")
                else:
                    print("❌ No artifacts found in response")
                    print(f"Response keys: {list(response_data.keys())}")
                    if 'error' in response_data:
                        print(f"Error from server: {response_data['error']}")
                        
            except json.JSONDecodeError as e:
                print(f"❌ Failed to parse JSON response: {e}")
                print(f"Raw response: {response_text}")

except subprocess.TimeoutExpired:
    print("❌ Request timed out (>5 minutes)")
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    import traceback
    traceback.print_exc()


## 🎉 That's it!

### To generate more 3D models:
1. Go back to the cell above with `PROMPT = "..."`
2. Change the prompt to whatever you want
3. Run both cells again

### Example prompts to try:
- `"A modern chair"`
- `"A futuristic car"`
- `"A wooden table"`
- `"A cozy bedroom"`
- `"A space station interior"`

### Useful commands:


In [ ]:
# Check if NIM container is running
!docker ps | grep nim-server


In [ ]:
# View recent container logs
!docker logs nim-server --tail 20


In [ ]:
# Stop the NIM container (when you're done)
!docker stop nim-server
!docker rm nim-server
print("🛑 Trellis NIM container stopped and removed")


In [ ]:
# List generated GLB files
!ls -la *.glb
